Tutorial Link:
https://www.mongodb.com/developer/products/atlas/advanced-rag-langchain-mongodb/

In [1]:
! pip install -qU datasets langchain langchain-mongodb langchain-openai pymongo pandas

In [2]:
import getpass
OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key:")

In [3]:
MONGODB_URI = getpass.getpass("Enter your MongoDB connection string:")

In [4]:
from datasets import load_dataset
import pandas as pd

data = load_dataset("MongoDB/embedded_movies")
df = pd.DataFrame(data["train"])

# Only keep records where the fullplot field is not null
df = df[df["fullplot"].notna()]

# Renaming the embedding field to "embedding" -- required by LangChain
df.rename(columns={"plot_embedding": "embedding"}, inplace=True)

Generating train split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [5]:
from pymongo import MongoClient

# Initialize MongoDB python client
client = MongoClient(MONGODB_URI)

DB_NAME = "langchain_chatbot"
COLLECTION_NAME = "data"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
collection = client[DB_NAME][COLLECTION_NAME]

In [6]:
# Delete any existing records in the collection
collection.delete_many({})

# Data Ingestion
records = df.to_dict('records')
collection.insert_many(records)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch

# Using the text-embedding-ada-002 since that's what was used to create embeddings in the movies dataset
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-ada-002")

# Vector Store Creation
vector_store = MongoDBAtlasVectorSearch.from_connection_string(
    connection_string=MONGODB_URI,
    namespace=DB_NAME + "." + COLLECTION_NAME,
    embedding= embeddings,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    text_key="fullplot"
)


In [8]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [9]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Generate context using the retriever, and pass the user question through
retrieve = {"context": retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])), "question": RunnablePassthrough()}
template = """Answer the question based only on the following context: \
{context}

Question: {question}
"""
# Defining the chat prompt
prompt = ChatPromptTemplate.from_template(template)
# Defining the model to be used for chat completion
model = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
# Parse output as a string
parse_output = StrOutputParser()

# Naive RAG chain 
naive_rag_chain = (
    retrieve
    | prompt
    | model
    | parse_output
)

In [11]:
naive_rag_chain.invoke("What is the best movie to watch when sad?")
# Output: 'Based on the context provided, "Smilla\'s Sense of Snow" would be the best movie to watch when sad as it invol

'Based on the context provided, "Smilla\'s Sense of Snow" would be the best movie to watch when sad as it involves a gripping mystery and conspiracy that could help take your mind off things.'

In [12]:
from langchain_mongodb.chat_message_histories import MongoDBChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder

def get_session_history(session_id: str) -> MongoDBChatMessageHistory:
        return MongoDBChatMessageHistory(MONGODB_URI, session_id, database_name=DB_NAME, collection_name="history")

In [13]:
standalone_system_prompt = """
Given a chat history and a follow-up question, rephrase the follow-up question to be a standalone question. \
Do NOT answer the question, just reformulate it if needed, otherwise return it as is. \
Only return the final standalone question. \
"""
standalone_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", standalone_system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

question_chain = standalone_question_prompt | model | parse_output

In [14]:
retriever_chain = RunnablePassthrough.assign(context=question_chain | retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])))

In [15]:
rag_system_prompt = """Answer the question based only on the following context: \
{context}
"""
rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rag_system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

In [16]:
# RAG chain
rag_chain = (
    retriever_chain
    | rag_prompt
    | model
    | parse_output
)

# RAG chain with history
with_message_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

In [18]:
with_message_history.invoke({"question": "What is the best movie to watch when sad?"}, {"configurable": {"session_id": "1"}})

# Output: The best movie to watch when feeling down could be "Last Action Hero." It\'s a fun and action-packed film that blends reality and fantasy, offering an escape from the real world and providing an entertaining distraction.

'The best movie to watch when sad would likely be "Smilla\'s Sense of Snow." It is a gripping mystery thriller that can help take your mind off things and immerse you in a suspenseful story, providing a distraction from your sadness.'

In [19]:
with_message_history.invoke({"question": "How about something more light?"}, {"configurable": {"session_id": "1"}})

# Output: For a lighter movie option, you might enjoy "Cousins." It\'s a comedy film set in Barcelona with action and humor, offering a fun and entertaining escape from reality. The storyline is engaging and filled with comedic moments that could help lift your spirits.

'For a lighter movie option, "48 Hrs." could be a good choice. It is a buddy cop comedy with action and humor that can help lift your spirits and provide some entertainment without being too heavy.'

In [20]:
from langchain_mongodb.cache import MongoDBAtlasSemanticCache
from langchain_core.globals import set_llm_cache

set_llm_cache(MongoDBAtlasSemanticCache(
    connection_string=MONGODB_URI,
    embedding=embeddings,
    collection_name="semantic_cache",
    database_name=DB_NAME,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    wait_until_ready=True # Optional, waits until the cache is ready to be used
))

In [21]:
%%time
naive_rag_chain.invoke("What is the best movie to watch when sad?")

CPU times: user 115 ms, sys: 36 ms, total: 151 ms
Wall time: 5.98 s


'Based on the context provided, "Smilla\'s Sense of Snow" would be the best movie to watch when sad as it involves a gripping mystery and conspiracy that could help take your mind off things.'

In [22]:
%%time
naive_rag_chain.invoke("Which movie do I watch when sad?")

CPU times: user 40.5 ms, sys: 23.9 ms, total: 64.4 ms
Wall time: 1.2 s


'Based on the context provided, "Smilla\'s Sense of Snow" would be the best movie to watch when sad as it involves a gripping mystery and conspiracy that could help take your mind off things.'